In [5]:
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

np.random.seed(42)


df = pd.read_csv('data/binance/BTCUSDT_1m_Futures.csv', index_col=0, parse_dates=True)
data = df[['Open', 'High', 'Low', 'Close', 'Volume']][-100000:]


data



,Open,High,Low,Close,Volume
Date,,,,,
2024-04-24 04:02:00,66432.5,66446.6,66415.3,66423.2,166.851
2024-04-24 04:03:00,66423.2,66423.3,66318.0,66403.0,864.803
2024-04-24 04:04:00,66402.9,66427.9,66380.0,66415.8,122.492
2024-04-24 04:05:00,66415.8,66416.3,66385.0,66416.3,89.957
2024-04-24 04:06:00,66416.2,66416.2,66300.0,66355.9,469.749
...,...,...,...,...,...
2024-07-02 14:37:00,62093.7,62093.7,62080.0,62093.0,39.111
2024-07-02 14:38:00,62092.9,62122.9,62086.6,62117.6,90.510
2024-07-02 14:39:00,62117.6,62142.0,62111.0,62142.0,63.159


In [42]:
from numba import jit, njit, prange, vectorize, float64, int64, cuda, float32, guvectorize, void
from numba import float32, int32, int64, void, float64
from numba.typed import List
from numba.types import UniTuple

import numba as nb


nb_float = float64
nb_int = int64



class Backtest:
    def __init__(self, data, ma_lenth=100, num_grid=100, fee=-0.00015, pyrimading=1, test_size=0.2, Rf= 8/365/24/60):
        
        self.Rf = Rf
        self.pyramiding = pyrimading
        self.fee = fee
        self.num_grid = num_grid

        
        stop_idx = data[(data['SMA'] < data['High']) & (data['SMA'] > data['Low'])].index[-1]
        data = data[:stop_idx].dropna()
        self.data = data
        
        self.split_idx = int(len(data) * (1 - test_size))
        
        self.decimal = len(str(data.iloc[0]['Close']).split('.')[1]) if '.' in str(data.iloc[0]['Close']) else 0
        self.data.loc[:, 'SMA'] = self.data['Close'].rolling(window=ma_lenth).mean().round(self.decimal)
        

        self.orders = self.get_orders(self.data.iloc[:self.split_idx], num_grid=num_grid)
        
        self.amount = np.ones((self.num_grid*2, self.pyramiding), dtype=np.float64)/(self.pyramiding*2*self.num_grid)*100
        
    @staticmethod
    def get_orders(data, num_grid=100):
        
        max_diff = ((data['SMA']-data['Close'])/data['SMA']).abs().max()

        id = np.concatenate([np.arange(-num_grid, 0, 1, dtype=int), np.arange(1, num_grid+1, dtype=int)])
        orders = {
            'id': id,
            'open_price': -id * max_diff/num_grid,
            'close_price': np.zeros(num_grid*2),
            'direction': np.sign(id),
            }

        orders = pd.DataFrame(orders)
        
        return orders

    @staticmethod
    # @njit([UniTuple(nb_float[:, :], 2)(nb_float[:, :], nb_float[:, :], nb_float, nb_float[:])], parallel=True, fastmath=True)
    @njit(parallel=True)
    def run_jit(data, orders, amount, fee):
        
        balance = np.zeros(len(data), dtype=np.float64)
        
        # pos = np.zeros((len(data), len(orders)), dtype=np.float64)
        
        # usdt_b = np.zeros(len(data), dtype=np.float64)
        
        normal_qty = 1 - fee
        
        for j in prange(len(orders)):
            
            id, open_on, close_on, direction = orders[j]
            
            pyramiding = amount[j]
            
            num_positions = 0
            
            usdt = 0
            
            crypto = 0

            for i in range(len(data)):
                open, high, low, close, volume, sma = data[i]
                open_price = (open_on +1 ) * sma
                close_price = (close_on +1 ) * sma
                
                if i == 0:
                    if direction > 0:
                        if open < open_price:
                            num_positions += 1
                            crypto += amount[j, 0]/open_price
                            usdt -= normal_qty * pyramiding[0]
                            # pos[i, j] = open_price
                            
                    else:
                        if open > open_price:
                            num_positions += 1
                            crypto -= amount[j, 0]/open_price
                            usdt += normal_qty * pyramiding[0]
                            # pos[i, j] = open_price

                for k in range(num_positions + 1):
                    if k == num_positions:

                        if direction > 0:

                            if low < open_price and open > open_price and k < len(pyramiding) - 1:

                                num_positions += 1
                                crypto += pyramiding[k]/open_price
                                usdt -= normal_qty * pyramiding[k]
                                # pos[i, j] = open_price

                        else:
                            if high > open_price and open < open_price and k < len(pyramiding) - 1:
                                num_positions += 1
                                crypto -= pyramiding[k]/open_price
                                usdt += normal_qty * pyramiding[k]
                                # pos[i, j] = open_price
                    else:
                        if direction > 0:
                            if high > close_price and open < close_price:
                                num_positions = 0
                                usdt += normal_qty * crypto * close_price
                                crypto = 0

                        else:    
                            if low < close_price and open > close_price:
                                num_positions = 0
                                usdt += normal_qty * crypto * close_price
                                crypto = 0
                
                # usdt_b[i] += usdt
                balance[i] += crypto * close + usdt
                              
        return balance, None, None
    
    
    def run(self):
        
        # positions = self.run_jit(self.data.to_numpy(), self.orders.to_numpy())

        # self.positions = positions
        self.balance, self.pos, self.usdt_b = self.run_jit(self.data.to_numpy(), self.orders.to_numpy(), self.amount, self.fee)
        self.drawdown = self.cummax_diff(self.balance)
        
        self.train_balance = self.balance[:self.split_idx]
        self.test_balance = self.balance[self.split_idx:]
        
        self.returns = np.diff(self.balance)
        
    def save(self, train_stats, val_stats, amount):
        import json
        with open('stats.json', 'w') as f:
            json.dump({
                'amount': amount.tolist(),
                'train_stats': train_stats,
                'val_stats': val_stats,

            }, f)
            
    
    @staticmethod
    def get_stats(returns, drawdown, Rf):
            
            mean = np.mean(returns)
            dev = np.abs(returns - mean).mean()
            std = np.std(returns)
            max_drawdown = -np.min(drawdown)
            
            sharpe = (mean - Rf) / std
            sharped = (mean - Rf) / dev
            calmar = (mean-Rf) / max_drawdown
            sortino = (mean - Rf) / np.std(returns[returns < 0])
            
            return mean, std, max_drawdown, sharpe, sharped, calmar, sortino
        
    def stats(self, print_stats=False):
        train_stats = self.get_stats(self.returns[:self.split_idx], self.drawdown[:self.split_idx], self.Rf)
        val_stats = self.get_stats(self.returns[self.split_idx:], self.drawdown[self.split_idx:]-self.drawdown[self.split_idx], self.Rf)
        if print_stats:
            print('Train', *train_stats)
            print('Test', *val_stats)
        # else:
        #     self.save(train_stats, val_stats, self.amount)
        return train_stats, val_stats
        

    @staticmethod
    @njit(parallel=True)
    def cummax_diff(A):
        B = np.zeros_like(A)
        max_val = A[0]
        for i in prange(len(A)):
            if A[i] < max_val:
                B[i] = A[i] - max_val
            else:
                max_val = A[i]
                B[i] = 0
        return B

    def test_jit(self):
    
        self.drawdown = self.portfolio.reshape(len(self.portfolio), -1).sum(axis=1)
        
    def plot(self):
        
        # plt.figure(figsize=(20, 10))
        # # plt.plot(self.data['Close'], label='Close')
        # plt.plot(self.data['SMA'], label='SMA')
        # # plt.plot(self.data['High'], label='High')   
        # # plt.plot(self.data['Low'], label='Low')
        
        # idx = np.where(self.pos != 0)
        # plt.scatter(self.data.iloc[idx[0]].index, self.pos[idx], c='m', s=4)
        # plt.legend()
        
        # plt.show()
        
        plt.figure(figsize=(20, 10))
        plt.plot(self.balance)
        plt.ylabel('gain in % of initial balance')
        plt.xlabel('Time')
        plt.title('Balance')
        plt.show()
        plt.figure(figsize=(20, 10))
        plt.plot(self.drawdown, label='Drawdown')
        plt.plot(np.zeros(self.drawdown.shape[0]), c='r')
        plt.show()
        # plt.figure(figsize=(20, 10))
        # plt.plot(self.usdt_b, label='USDT balance')
        # plt.show()

bt = Backtest(data, pyrimading=20)

bt.run()
bt.stats()

((np.float64(1.0187045794043376e-06),
  np.float64(0.0021286337770640164),
  np.float64(0.4865818515663791),
  np.float64(-0.006671883029306808),
  np.float64(-0.021137225160803304),
  np.float64(-2.9187269371194865e-05),
  np.float64(-0.006559781358145942)),
 (np.float64(3.594564909359554e-06),
  np.float64(0.0014455636070736773),
  np.float64(0.1311278301072818),
  np.float64(-0.008042631390245624),
  np.float64(-0.024761693574376756),
  np.float64(-8.866260681150267e-05),
  np.float64(-0.008408884119342425)))

In [28]:
%timeit -n 10 -r 7 bt.run()

21.6 ms ± 3.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [43]:
%timeit -n 10 -r 7 bt.stats()

1.17 ms ± 345 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
